In [ ]:
"""
Welcome to our data collection Python notebook for testing news asymmetry between small and large companies!
Run the cells from the top, all the way down the page by clicking on the first cell, then holding down
shift or command or control, then hitting enter
"""

#Install the modules that you will need for reading in all of the data:

#Uncomment the following two lines the first time you run this in order to install the necessary modules

#!pip install pandas-datareader
#!pip install yahoo-finance

In [ ]:
#Import the following modules:
import yahoo_finance as yfin
import pandas as pd
import pandas_datareader.data as web
import datetime
import csv
import pandas_datareader.data as data
from yahoo_finance import Share
from pandas_datareader.yahoo.quotes import _yahoo_codes

In [ ]:
#'Jumps' in data--a place for our ideas
"""
Defining a Jump
When an event happens
Positive/Negative Event
Positive/Negative interest
"""
#trends-->prices
#prices-->trends
#volume-->prices

In [ ]:
#read in the csv with the russell 3000 stock data and Wilshire 5000 stock data (Just a dataframe of symbols for now)
dfRussell=pd.read_csv('russell_3000_2011-06-27.csv')
dfWilshire=pd.read_csv('wilshire5000.csv')

In [ ]:
#Taking the values from the dataframes and making them into arrays
arrayRussell=dfRussell.values.flatten()
arrayWilshire=dfWilshire.values.flatten()

In [ ]:
yahoo=Share('YHOO')

In [ ]:
print yahoo

In [ ]:
yahoo.get_price()

In [11]:
start=datetime.datetime(2013,1,1)
end=datetime.datetime(2017,1,1)
df = data.DataReader(arrayWilshire[0:4], 'yahoo', start, end) 
type(df)
#dates =[]
#for x in range(len(df)):
#    newdate = str(df.index[x])
#    newdate = newdate[0:10]
#    dates.append(newdate)

#df['dates'] = dates

#print df.head()
#print df.tail()

pandas.core.panel.Panel

In [14]:
stocks = ['ORCL', 'TSLA', 'IBM','YELP', 'MSFT']
ls_key = 'Adj Close'
start = datetime.datetime(2000,1,1)
end = datetime.datetime(2016,11,28)
f = data.DataReader(arrayOfStocks[:50], 'yahoo',start,end)

cleanData = f.ix[ls_key]
dataFrame = pd.DataFrame(cleanData)

print dataFrame[:]

NameError: name 'data' is not defined

In [15]:
stocklist = ['aapl','goog','fb','amzn','COP']

#http://www.jarloo.com/yahoo_finance/
#https://greenido.wordpress.com/2009/12/22/yahoo-finance-hidden-api/
_yahoo_codes.update({'Market Cap': 'j1'})
_yahoo_codes.update({'Div Yield': 'y'})
_yahoo_codes.update({'Bid': 'b'})
_yahoo_codes.update({'Ask': 'a'})
_yahoo_codes.update({'Prev Close': 'p'})
_yahoo_codes.update({'Open': 'o'})
_yahoo_codes.update({'1 yr Target Price': 't8'})
_yahoo_codes.update({'Earnings/Share': 'e'})
_yahoo_codes.update({"Day’s Range": 'm'})
_yahoo_codes.update({'52-week Range': 'w'})
_yahoo_codes.update({'Volume': 'v'})
_yahoo_codes.update({'Avg Daily Volume': 'a2'})
_yahoo_codes.update({'EPS Est Current Year': 'e7'})
_yahoo_codes.update({'EPS Est Next Quarter': 'e9'})

data.get_quote_yahoo(stocklist).to_csv('test.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

NameError: name '_yahoo_codes' is not defined

In [17]:
data.get_quote_yahoo(stocklist).transpose()

NameError: name 'data' is not defined